In [1]:
## Laod all models
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
# Load all the pickle file
model = load_model('model.h5')

# load encoder, gender and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('one_hot_encoder.pkl','rb') as file:
    one_hot_encoder = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [3]:
# Example data
input_data = {
    'credit_score':600,
    'country':'Spain',
    'gender':'Female',
    'age':42,
    'tenure':3,
    'balance':60000,
    'products_number':3,
    'credit_card':1,
    'active_member':1,
    'estimated_salary':50000
}

In [5]:
# Encode the country, gender
geo_encoded = one_hot_encoder.transform([[input_data['country']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=one_hot_encoder.get_feature_names_out(['country']))
geo_encoded_df

D:\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,country_France,country_Germany,country_Spain
0,0.0,0.0,1.0


In [6]:
# convert key-value pairs into dataframe
input_df = pd.DataFrame([input_data])


In [7]:
# Encode categorical variables
input_df['gender']=label_encoder_gender.transform(input_df['gender'])
input_df

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary
0,600,Spain,0,42,3,60000,3,1,1,50000


In [8]:
# Concat One hot encoded data to input_df
input_df = pd.concat([input_df.drop('country', axis=1), geo_encoded_df], axis=1)
input_df

,credit_score,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,country_France,country_Germany,country_Spain
0,600,0,42,3,60000,3,1,1,50000,0.0,0.0,1.0


In [9]:
# Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.47154541, -1.09997489,  0.28778948, -0.69844549, -0.29010416,
         2.50185008,  0.63367318,  0.95214374, -0.84805047, -1.02020406,
        -0.57581067,  1.77464858]])

In [10]:
# Predict the churn using the model
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([[0.99474424]], dtype=float32)

In [11]:
prediction_prob = prediction[0][0]
if prediction_prob >0.5:
    print("Customer is likely to exit")
else:
    print("Customer is not likely to exit")


Customer is likely to exit
